In [23]:
import torch
assert torch.__version__ == "1.4.0"
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torch.nn.functional as F

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np


In [13]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cpu


In [14]:
!pwd
def load_dataset():
    data_path = './data-resized'
    train_dataset = dset.ImageFolder(
        root=data_path,
        transform=T.ToTensor()
    )
    train_loader = DataLoader(
        train_dataset,
        batch_size=64,
        num_workers=0,
        shuffle=True
    )
    return train_loader

/Users/michaeldu/Desktop/StanfordCS/cs231n/DeepAdDetection


In [ ]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [26]:
class Flatten(nn.Module):
    def forward(self, x):
        N = x.shape[0] # read in N, C, H, W
        return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image


def train(model, optimizer, epochs=1):
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        print(e,epochs)
        for t, (x, y) in enumerate(load_dataset()):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            print(x.shape)
            scores = model(x)
            print(scores, y)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
#                 check_accuracy(loader_val, model)
#                 print()
                

In [28]:

model = None
optimizer = None

model = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=(5,5), padding=2),
# #     Flatten(),
#     nn.Linear(256, 1),
#     nn.Softmax()
)


optimizer = torch.optim.Adam(model.parameters(), lr=.9e-3, weight_decay=1e-5)

train(model, optimizer, epochs=1)

0 1
torch.Size([64, 3, 256, 256])
tensor([[[[-1.7282e-02, -7.0621e-02, -8.1615e-02,  ..., -4.5044e-01,
           -2.6015e-01, -1.8870e-01],
          [-5.3459e-03, -4.8873e-02, -5.9538e-02,  ..., -4.0332e-01,
           -2.1032e-01, -3.3603e-02],
          [ 7.3673e-03, -4.0330e-02, -4.7501e-02,  ..., -3.6667e-01,
           -2.4026e-01, -1.2910e-01],
          ...,
          [-2.1408e-01, -3.9578e-01, -4.3905e-01,  ..., -4.9729e-01,
           -3.3041e-01, -1.8619e-01],
          [-6.8659e-02, -2.4684e-01, -2.4863e-01,  ..., -2.8026e-01,
           -1.9133e-01, -1.1827e-01],
          [-9.1619e-02, -2.1954e-01, -2.0982e-01,  ..., -2.4197e-01,
           -1.1265e-01, -5.6403e-02]],

         [[ 1.0113e-01,  2.6338e-02,  9.2904e-03,  ..., -2.2232e-01,
           -2.0049e-01, -9.1406e-02],
          [ 1.3795e-01,  6.6844e-02,  2.0015e-02,  ..., -1.7793e-01,
           -2.1501e-01, -2.2534e-01],
          [ 2.1577e-01,  1.4078e-01,  6.7859e-02,  ..., -5.9058e-03,
           -1.8347e-01, 

RuntimeError: only batches of spatial targets supported (3D tensors) but got targets of dimension: 1

NameError: name 'tqdm' is not defined